In [1]:
from icecube import dataclasses, icetray, dataio
import numpy as np
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

### Frame structure of IceTray

By default, IceCube software that is base for our development has the following types of frames:
  - Simulation (S)
  - Geometry (G)
  - Calibration (C)
  - DetectorStatus (D)
  - DAQ (Q)
  - Physics (P)

This also defines hierarchy of frames. Typically,content of previous frames of different type can be accessed inside the next frames, unless new frame contains data with the same key, in what case new object is available. 

Special case is made for hierarchy of Q and P frames. When reader finds a new Q frame, the content of P-frame is not available. While subsequent P-frames have access to data of previous Q-frame. The main reason for that is possible frame splitting, when 1 Q frame (corresponding to DAQ reading) can be split into different events, each stored in individual P-frames. Typically frames of higher order are referred as "parent" frame, for example, Q frame is parent of P frame.

Now, let's open an example file and look inside the frames.

In [2]:
infile = dataio.I3File("small_test_file.i3.bz2")

Each file is effectively a stream of individual indendent frames. To get the next frame in the event, one has to call `pop_frame()`, if you are interested in Q or P frame, one can use dedicated `pop_daq()` and `pop_physics()` functions. Important to note that this function gets next frame, making previous frame not accessible (unless it's data from parent frame). Let's get a P-frame and look inside:

In [3]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Geometry):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
]



In [4]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Calibration):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
]



In [5]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Simulation):
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [6]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (DAQ):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [7]:
frame = infile.pop_frame()
print(frame)

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]



In [8]:
print(frame)
# hethis one isan example of double stored in P frame
print(frame['PhysParameter'])
# here is a vector of floats 
print(frame[f'DAQObject{frame["PFrameNumber"].value}'])
###

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]

I3Double(504.94)
[997, 839, 34, 799, 89, 416, 370, 882, 619, 623, 966, 827, 466, 742, 627, 437, 987, 504, 63, 554, 645, 865, 574, 812, 805, 432, 133, 634, 983, 113, 978, 337, 251, 55, 439, 447, 676, 22, 599, 10, 549, 117, 326, 783, 341, 413, 668, 231, 321, 64, 867, 662, 584, 638, 577, 88, 994, 658, 282, 509, 227, 90, 631, 136, 378, 597, 189, 87, 640, 209, 125, 198, 854, 388, 782, 224, 694, 478, 753, 316, 464, 948, 950, 787, 826, 916, 23, 188, 345, 545, 301, 247, 184, 181, 881, 904, 984, 266, 915, 320]


Now let's get the next P-frame (number 2) that contains avearage of `DAQObject1`

In [9]:
frame = infile.pop_frame()
print(frame)
# hethis one is an example of double stored in P frame
print(frame['PhysParameter'], frame["PFrameNumber"].value)
# here is a vector of floats 
print(frame[f'DAQObject{frame["PFrameNumber"].value}'])

[ I3Frame  (Physics):
  'DAQObject1' [DAQ] ==> I3Vector<double> (838)
  'DAQObject2' [DAQ] ==> I3Vector<double> (838)
  'PFrameNumber' [Physics] ==> I3PODHolder<int> (29)
  'PhysParameter' [Physics] ==> I3PODHolder<double> (36)
  'SomeNumberOfStrings' [Geometry] ==> I3PODHolder<int> (29)
  'SomeRQECalibration' [Calibration] ==> I3Map<OMKey, double> (65)
  'SomeSimParameter' [Simulation] ==> I3PODHolder<double> (36)
]

I3Double(-4014.21) 2
[-4529, -4976, -4808, -4056, -3146, -3083, -3140, -4475, -4659, -3372, -3899, -4079, -4001, -3968, -3883, -4342, -4190, -4502, -3594, -3086, -3329, -3517, -3842, -3103, -3842, -4341, -4433, -4481, -3532, -4826, -4043, -4061, -3248, -4063, -3744, -3229, -4354, -3267, -3155, -4040, -3555, -4779, -4151, -4442, -4508, -4418, -4127, -3217, -3425, -4604, -4417, -3430, -4113, -4929, -3633, -4345, -4705, -4561, -3860, -3654, -4859, -4944, -3291, -3195, -4869, -4119, -4084, -3098, -3960, -3846, -3229, -4486, -3582, -3424, -3802, -4016, -4405, -4448, -3240, -31

As one can see, the frame looks close to a map/dictionary holding serialized data. In printout one can also see the type of the frame which contains certain inforamtio (for example `[DAQ]` or `[Physics]` ). The frame is agnostic to type of data it holds and supports different data types. Most of the available datatypes can be found in dataclasses object (except of `I3Int`, `I3Bool`, which is part of icetray due to historic reasons), plus in individual projects. Standard python or C++ object has to be first converted to icetray object to be stored, as it requires custom serialization. 